## Introduction

This project explores an unsupervised clustering problem on the PatternMind dataset: 25k+ images spanning 233 fine-grained classes. Rather than working with raw pixels, we convert images into CNN feature vectors to analyze how they naturally group in this learned feature space. Ground-truth labels are used only to evaluate results and establish baselines. Our primary goal is to uncover the dataset's visual structure: which clusters emerge organically, how broader macro-groupings form, and where clustering reaches its limits.

Our pipeline begins with data inspection (class imbalance, image dimensions), then trains a CNN primarily as a feature extractor, using its 256-D penultimate layer as a compact embedding for each image. We standardize these embeddings and apply PCA to reduce redundancy (50D for clustering, 2D for visualization). Hierarchical clustering on category centroids reveals data-driven macro-categories that capture visual affinities at multiple granularities (5–30 groups).

We evaluate K-Means clustering by measuring purity against both the original 233 labels and our derived macro-categories. A supervised Logistic Regression classifier on the same embeddings provides an upper bound, quantifying the gap between unsupervised clustering and label classification.

## 1. Data Loading and Initial Exploration

In [ ]:
import os
import numpy as np
import pandas as pd
from keras import regularizers
from PIL import Image
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import keras
from collections import Counter
from keras import layers
import tensorflow as tf
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from scipy.cluster.hierarchy import linkage, fcluster, leaves_list
from scipy.spatial.distance import pdist, squareform

### 1.1 Load Image Paths and Labels

We load all images from the dataset directory, where each subfolder represents a distinct category. The code below creates a DataFrame containing the file path, category name, and numeric label for each image.

In [ ]:
dataset_path = "db/patternmind_dataset/"

# Get all category folders (each subfolder is a class)
categories = sorted([d for d in os.listdir(dataset_path) if os.path.isdir(
    os.path.join(dataset_path, d)) and not d.startswith('.')])

# Build a list of all images with their metadata
data = []
for label_id, category in enumerate(categories):
    folder_path = os.path.join(dataset_path, category)
    images = [f for f in os.listdir(
        folder_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

    for img in images:
        data.append({
            'path': os.path.join(folder_path, img),
            'category': category,
            'label_id': label_id
        })

df = pd.DataFrame(data)

### 1.2 Dataset Statistics and Quality Check

The code below checks for missing values, duplicates and computes summary statistics about the class distribution.

In [ ]:
# Check for missing values and data quality
print(f"Missing values in DataFrame: {df.isnull().sum().sum()}")
print(f"Duplicate rows: {df.duplicated().sum()}")

# Detailed statistics
counts = df['category'].value_counts()
print(f"\nTotal images: {len(df)}")
print(f"Total categories: {len(categories)}")
print(f"Average images per category: {len(df) / len(categories):.2f}")
print(f"Min images per category: {counts.min()}")
print(f"Max images per category: {counts.max()}")
print(f"Median images per category: {counts.median()}")
print(f"Std deviation: {counts.std():.2f}")
print("\nEdge cases (fewest images):")
print(counts.tail())
print("\nEdge cases (most images):")
print(counts.head())

# Class imbalance analysis
imbalance_ratio = counts.max() / counts.min()
print(f"\nClass imbalance ratio (max/min): {imbalance_ratio:.2f}")

The output above shows:

- No missing values or duplicates
- 25557 total images across 233 categories
- Average of ~110 images per category, but with significant variation
- Class imbalance ratio of 10.57x between the most and least frequent categories

## 2. Exploratory Data Analysis (EDA)

### 2.1 Class Distribution Analysis

We first examine how images are distributed across categories.

In [ ]:
fig_dist = px.bar(counts, x=counts.index, y=counts.values,
                  title="Class Distribution",
                  labels={'index': 'Category', 'y': 'Count'})
fig_dist.update_layout(xaxis={'categoryorder': 'total descending'})
fig_dist.show()

The bar chart reveals significant class imbalance in the dataset:

- The most frequent category ("clutter") has 761 images
- The least frequent category ("top-hat") has only 72 images
- This represents a 10x variation in class sizes

This imbalance matters for both supervised classification and unsupervised clustering. During CNN training we use data augmentation to help the model generalize across all categories regardless of frequency. For clustering we report purity, which is weighted by cluster size, so large clusters influence the score more than small ones.

### 2.2 Image Dimension Analysis

Before feeding images to a neural network, we need to understand the variation in image sizes. Neural networks require fixed-size inputs, so we must choose an appropriate target resolution. We analyze a sample of 5,000 images to understand the distribution of widths and heights.

In [ ]:
# Sample images to analyze dimension variability
sample_size = 5000
sample_df = df.sample(n=sample_size)
sizes = [Image.open(p).size for p in sample_df['path']]
widths, heights = zip(*sizes)

# Create side-by-side histograms for width and height distributions
fig_sizes = make_subplots(rows=1, cols=2, subplot_titles=(
    "Width Distribution", "Height Distribution"))
fig_sizes.add_trace(go.Histogram(x=widths, name="Width"), row=1, col=1)
fig_sizes.add_trace(go.Histogram(x=heights, name="Height"), row=1, col=2)
fig_sizes.update_layout(
    title_text=f"Image Size Distribution (Sample n={sample_size})")
fig_sizes.show()

The histograms show that image dimensions vary considerably:

- Most images have widths and heights concentrated around 200-400 pixels
- Some images are very small (thumbnails) while others are much larger

We chose 224×224 resolution (the ImageNet standard) because it is large enough to preserve important visual details like edges, textures, and shapes, while remaining computationally efficient by reducing memory usage and training time.

### 2.3 Visual Sample Inspection

To understand the visual diversity in our dataset we display random samples from five different categories. This helps us appreciate the challenges our models will face: varying backgrounds, lighting conditions, object orientations and image quality.

In [ ]:
sample_categories = np.random.choice(categories, 5, replace=False)
fig_imgs = make_subplots(rows=1, cols=5, subplot_titles=sample_categories)

for i, cat in enumerate(sample_categories):
    img_path = df[df['category'] == cat].sample(1).iloc[0]['path']
    img = Image.open(img_path)
    img_array = np.array(img)
    fig_imgs.add_trace(go.Image(z=img_array), row=1, col=i+1)

fig_imgs.update_layout(title_text="Sample Images from Random Categories")
fig_imgs.show()

The random samples reveal substantial visual diversity within and across categories.

To address this variability, we use data augmentation (random flips, rotations, zoom, translations) so the model sees different orientations and scales of the same objects. The CNN’s stacked convolutional layers then learn hierarchical features that stay robust to these variations.

## 3. CNN Feature Extraction

We use a CNN for feature extraction because these networks learn representations at multiple levels of complexity. Early layers detect simple patterns like edges and textures while deeper layers recognize more complex features like shapes and objects. The layer just before the final classification output contains a rich and compressed representation of the image that captures its essential visual characteristics. These learned features work much better for clustering than using raw pixel values directly.

### 3.1 Data Preprocessing and Augmentation

Before training the CNN, we standardize inputs and apply light augmentation:

- Resize to 224×224: fixed input shape
- Normalize to [0, 1]: scale pixels for faster convergence
- Random horizontal flip: handle left/right orientation
- Random rotation (±0.15 rad): cover tilt
- Random zoom (±15%): cover scale changes
- Random translation (±10% height/width): cover shifts

We split 80% for training and 20% for validation.

In [ ]:
# type: ignore
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Data augmentation: random flips, rotations, zoom, and translations
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.15),
    layers.RandomZoom(0.15),
    layers.RandomTranslation(0.1, 0.1),
])

# Load training and validation datasets (80/20 split)
train_ds = keras.utils.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

val_ds = keras.utils.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

# Apply augmentation to training data and normalize both datasets
train_ds = train_ds.map(lambda x, y: (x / 255.0, y))
train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y))
train_ds = train_ds.cache().shuffle(1000).prefetch(tf.data.AUTOTUNE)

val_ds = val_ds.map(lambda x, y: (x / 255.0, y))
val_ds = val_ds.cache().prefetch(tf.data.AUTOTUNE)

### 3.2 CNN Model Architecture

We design a custom CNN with three convolutional blocks to extract visual features from images.

| Layer Type    | Details                                                              | Purpose                                        |
| ------------- | -------------------------------------------------------------------- | ---------------------------------------------- |
| Conv Block 1  | 32 filters × 2 layers, 3×3 kernel, SAME pad, L2=1e-4                 | Detect low-level features (edges, colors)      |
| Conv Block 2  | 64 filters × 2 layers, 3×3 kernel, SAME pad, L2=1e-4 + Dropout 0.25  | Detect mid-level features (textures, patterns) |
| Conv Block 3  | 128 filters × 2 layers, 3×3 kernel, SAME pad, L2=1e-4 + Dropout 0.25 | Detect high-level features (shapes, parts)     |
| GlobalAvgPool |                                                                      | Aggregate spatial features                     |
| Dense Layer   | 256 neurons, ReLU, L2=1e-4 + Dropout 0.5                             | Compact 256-D representation for clustering    |
| Output Layer  | 233 neurons, softmax                                                 | Classify into one of 233 categories            |

Key choices:

- Activation: ReLU throughout convolutional and dense layers.
- Padding: `padding="same"` keeps spatial size, preserving edges.
- Regularization: L2 weight decay `1e-4` on all Conv/Dense layers.
- Pooling: MaxPooling halves spatial dimensions in each block; GlobalAveragePooling before Dense.
- Dropout: 0.25 after pooling in blocks 2–3; 0.5 before the output layer.
- Batch size: 32, balancing stability and memory.
- Optimizer: Adam with learning rate 0.0005.
- Loss: Categorical crossentropy.
- Callbacks: Early stopping on `val_loss` (patience 5, restore best) and ReduceLROnPlateau (factor 0.5, patience 3, min_lr 1e-7).

Our goal isn’t peak classification accuracy, we primarily use the 256-D Dense layer as the feature embedding for clustering.

In [ ]:
MODEL_PATH = 'out/models/cnn_feature_extractor_v3.keras'

if os.path.exists(MODEL_PATH):
    model = keras.models.load_model(MODEL_PATH)
    print("Model loaded successfully!")
else:
    # Build CNN with 3 convolutional blocks (32, 64, 128 filters)
    model = keras.Sequential([
        layers.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),

        # Block 1
        layers.Conv2D(32, (3, 3), activation='relu', padding='same',
                      kernel_regularizer=regularizers.l2(1e-4)),
        layers.Conv2D(32, (3, 3), activation='relu', padding='same',
                      kernel_regularizer=regularizers.l2(1e-4)),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),

        # Block 2
        layers.Conv2D(64, (3, 3), activation='relu', padding='same',
                      kernel_regularizer=regularizers.l2(1e-4)),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same',
                      kernel_regularizer=regularizers.l2(1e-4)),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),

        # Block 3
        layers.Conv2D(128, (3, 3), activation='relu', padding='same',
                      kernel_regularizer=regularizers.l2(1e-4)),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same',
                      kernel_regularizer=regularizers.l2(1e-4)),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),

        layers.GlobalAveragePooling2D(),

        layers.Dense(256, activation='relu', name='features',
                     kernel_regularizer=regularizers.l2(1e-4)),
        layers.Dropout(0.5),
        layers.Dense(len(categories), activation='softmax')
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.0005),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    # Training callbacks for early stopping and learning rate reduction
    callbacks = [
        keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        ),
        keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=3,
            min_lr=1e-7
        )
    ]

    history = model.fit(
        train_ds,
        epochs=50,
        validation_data=val_ds,
        callbacks=callbacks
    )

    # Save the trained model
    os.makedirs('out/models', exist_ok=True)
    model.save(MODEL_PATH)
    print(f"Model saved to {MODEL_PATH}")

| Epoch | Accuracy   | Loss      | Val Accuracy | Val Loss  | Learning Rate |
| ----- | ---------- | --------- | ------------ | --------- | ------------- |
| 1     | 7.09%      | 5.151     | 6.05%        | 5.081     | 0.00050       |
| 5     | 14.71%     | 4.347     | 12.85%       | 4.558     | 0.00050       |
| 10    | 20.88%     | 3.841     | 14.46%       | 4.479     | 0.00050       |
| 15    | 26.52%     | 3.498     | 20.00%       | 3.997     | 0.00050       |
| 20    | 32.46%     | 3.111     | 26.18%       | 3.656     | 0.00025       |
| 25    | 35.62%     | 2.930     | 24.87%       | 3.726     | 0.00025       |
| 30    | 38.10%     | 2.778     | 26.57%       | 3.689     | 0.00025       |
| 33    | 40.91%     | 2.624     | 30.23%       | 3.409     | 0.000125      |
| 37    | **43.06%** | **2.501** | **30.33%**   | **3.445** | **0.0000625** |
| 38    | 43.26%     | 2.496     | 30.09%       | 3.470     | 0.0000625     |

The model trained for 38 epochs total, with the final weights saved at the end. The ReduceLROnPlateau callback automatically cut the learning rate in half three times: first at epoch 20 (from 5e-4 to 2.5e-4), again at epoch 33 (to 1.25e-4), and finally at epoch 37 (to 6.25e-5). This gradual slowdown helped the model fine-tune its weights more carefully as training progressed.

Looking at performance, the model hit its best validation accuracy of 30.3% at epoch 37 and its lowest validation loss of 3.41 at epoch 33. By the final epoch, training accuracy reached 43.3% while validation sat at 30.1%: a gap of about 13 percentage points. This gap shows the model is overfitting somewhat.

That said, 30% validation accuracy on 233 categories isn't terrible (random guessing would only get 0.43%). More importantly, we're not really after high classification scores here. What matters is that the network learned useful visual patterns in those intermediate layers and the 256-dimensional feature embedding it produces is what we'll actually use for clustering downstream.

### 3.3 Feature Extraction from All Images

Now we extract 256-dimensional feature vectors from every image in the dataset. We do this by:

1. Removing the final classification layer from the trained CNN
2. Passing each image through the network
3. Collecting the output from the Dense(256) layer

These features capture the "essence" of each image as learned by the CNN: a compact representation that preserves visual similarity. Images that look alike will have similar feature vectors making them suitable for clustering.

In [ ]:
FEATURES_PATH = 'out/features/features_v3.npy'
LABELS_PATH = 'out/features/labels_v3.npy'

if os.path.exists(FEATURES_PATH) and os.path.exists(LABELS_PATH):
    features = np.load(FEATURES_PATH)
    labels = np.load(LABELS_PATH)
    print(f"Features loaded: {features.shape}")
    print(f"Labels loaded: {labels.shape}")
else:
    # Create feature extractor by removing final classification layer
    feature_extractor = keras.Sequential(model.layers[:-1])  # type: ignore

    # Load all images without augmentation
    all_ds = keras.utils.image_dataset_from_directory(
        dataset_path,
        label_mode='int',
        shuffle=False,
        image_size=IMG_SIZE,
        batch_size=BATCH_SIZE
    )

    all_ds = all_ds.map(lambda x, y: (x / 255.0, y))  # type: ignore

    # Extract 256-dimensional features for all images
    features = feature_extractor.predict(all_ds.map(lambda x, y: x))
    labels = np.concatenate([y.numpy() for _, y in all_ds], axis=0)

    # Save features and labels
    os.makedirs('out/features', exist_ok=True)
    np.save(FEATURES_PATH, features)
    np.save(LABELS_PATH, labels)
    print(f"Features extracted and saved:")
    print(f"Features shape: {features.shape}")
    print(f"Labels shape: {labels.shape}")

## 4. Dimensionality Reduction with PCA

Our CNN outputs 256-dimensional feature vectors for each image, but working with that many dimensions has drawbacks. In high-dimensional spaces distances become less meaningful, clustering algorithms slow down and visualizations become impossible. Plus, the CNN's 256 features likely contain redundancy: some dimensions might encode similar information or just noise.

Principal Component Analysis (PCA) helps by finding new axes (principal components) that capture the maximum variance in our data. We can then keep only the most important components, throwing away redundant or noisy dimensions while preserving the essential structure. This makes clustering faster and more effective and lets us create 2D visualizations to understand what the CNN learned.

### 4.1 Feature Standardization and Variance Analysis

We standardize all 256 CNN features to zero mean and unit variance then fit PCA to analyze how variance is distributed across components. This tells us how many dimensions actually carry useful information versus redundancy or noise. The cumulative variance plot below guides our choice of how many components to retain for downstream clustering.

In [ ]:
features_loaded = np.load(FEATURES_PATH)
labels_loaded = np.load(LABELS_PATH)

# Standardize features (mean=0, std=1)
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features_loaded)

# Fit PCA to analyze variance distribution
pca_full = PCA()
pca_full.fit(features_scaled)

explained_variance = pca_full.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance)

fig_variance = go.Figure()
fig_variance.add_trace(go.Scatter(
    x=list(range(1, len(explained_variance) + 1)),
    y=cumulative_variance,
    mode='lines+markers',
    name='Cumulative Variance'
))
fig_variance.update_layout(
    title='PCA Explained Variance',
    xaxis_title='Number of Components',
    yaxis_title='Cumulative Explained Variance',
    hovermode='x'
)
fig_variance.show()

print(
    f"Variance explained by first 2 components: {cumulative_variance[1]:.2%}")
print(
    f"Variance explained by first 10 components: {cumulative_variance[9]:.2%}")
print(
    f"Variance explained by first 50 components: {cumulative_variance[49]:.2%}")
print(
    f"Variance explained by first 60 components: {cumulative_variance[59]:.2%}")
print(
    f"Variance explained by first 75 components: {cumulative_variance[74]:.2%}")
print(
    f"Variance explained by first 100 components: {cumulative_variance[99]:.2%}")
print(
    f"Variance explained by first 120 components: {cumulative_variance[119]:.2%}")

| Components | Variance Explained |
| ---------- | ------------------ |
| First 2    | 15.82%             |
| First 10   | 50.51%             |
| First 50   | 85.06%             |
| First 60   | 87.31%             |
| First 75   | 89.88%             |
| First 100  | 92.95%             |
| First 120  | 94.71%             |

The variance distribution shows that the CNN's 256-dimensional output contains substantial redundancy.

- The first 10 components alone capture over half the variance (50.51%)
- The first 50 components retain 85% of the information
- Beyond 100 components, variance gains diminish significantly
- The curve shows smooth accumulation without a sharp elbow, indicating that variance is distributed across many components rather than concentrated in a few

We choose 50 components for downstream clustering. This configuration retains 85.06% of the total variance while reducing dimensionality by over 80% compared to the original 256 dimensions.

### 4.2 2D Visualization

While 50 dimensions is optimal for clustering, we cannot visualize 50D data directly. We project features to just 2 dimensions (PC1 and PC2) to create scatter plots that give us intuition about the data structure. We visualize 5 categories selected for their semantic diversity: airplanes, saturn, mars, duck and goose. Note that this 2D projection captures only ~16% of the total variance, so some overlap is expected.

In [ ]:
# Reduce to 50D for clustering
pca_50d = PCA(n_components=50)
features_50d = pca_50d.fit_transform(features_scaled)

# Reduce to 2D for visualization
pca_2d = PCA(n_components=2)
features_2d = pca_2d.fit_transform(features_scaled)

# Visualize semantically distinct categories
distinct_categories = ['airplanes', 'saturn', 'mars', 'duck', 'goose']
distinct_indices = [i for i, cat in enumerate(
    categories) if cat in distinct_categories]

mask = np.isin(labels_loaded, distinct_indices)
df_pca_distinct = pd.DataFrame({
    'PC1': features_2d[mask, 0],
    'PC2': features_2d[mask, 1],
    'category': [categories[label] for label in labels_loaded[mask]]
})

fig_pca_distinct = px.scatter(
    df_pca_distinct,
    x='PC1',
    y='PC2',
    color='category',
    title='PCA 2D Projection - Semantically Distinct Categories',
    opacity=0.7
)
fig_pca_distinct.update_traces(marker=dict(size=5))
fig_pca_distinct.update_layout(width=1000, height=800)
fig_pca_distinct.show()

The scatter plot reveals clear separation between semantically distinct categories. Airplanes (blue) form a tight, well-defined cluster in the upper-left region, demonstrating that the CNN learned distinctive features for this category.

The space-related categories show interesting behavior: mars (purple) and saturn (orange) occupy the center region and overlap substantially with each other. This makes sense visually since both are celestial bodies with similar color palettes (warm oranges/reds) and spherical shapes. The CNN's features capture this shared "space object" signature.

Similarly, the bird categories duck (orange) and goose (green) cluster together in the right portion of the plot, reflecting their shared visual characteristics: feathers, beaks, similar body shapes, and often similar backgrounds (water, grass).

The key insight is that even in just 2 dimensions (capturing only ~16% of variance), the CNN features successfully separate semantically different concepts (vehicles vs. space objects vs. birds) while grouping visually similar categories together. This confirms that the learned representations encode meaningful visual structure that hierarchical and K-Means clustering can exploit in the full 50-dimensional space.

### 4.3 Data-Driven Macro-Category Discovery

Rather than imposing predefined semantic groupings (e.g., "animals," "vehicles"), we let the CNN features reveal natural visual clusters. We apply hierarchical clustering to the 233 category centroids using Ward linkage, which minimizes within-cluster variance at each merge step. The resulting dendrogram shows how categories progressively combine as we relax similarity thresholds.

Categories merging at low heights are visually similar according to the CNN's learned representations, while those merging only at high heights are fundamentally different. By cutting the dendrogram at various heights, we can create different numbers of macro-categories (from 5 broad groups to 30+ finer divisions) for use in our clustering evaluation in Section 6.

In [ ]:
# Compute centroids for each of the 233 categories
# A centroid is the mean feature vector of all images in that category
category_centroids = []
for i, category in enumerate(categories):
    mask = labels == i
    centroid = features_50d[mask].mean(axis=0)
    category_centroids.append(centroid)

category_centroids = np.array(category_centroids)

# Perform hierarchical clustering on category centroids using Ward linkage
# Ward linkage minimizes within-cluster variance at each merge step
linkage_matrix = linkage(category_centroids, method='ward')

# Create dendrogram with Plotly
color_threshold = 0.5 * max(linkage_matrix[:, 2])  # ~50% of max height

fig_dendro_analysis = ff.create_dendrogram(
    category_centroids,
    labels=categories,
    linkagefun=lambda x: linkage_matrix,
    color_threshold=color_threshold
)

# Add horizontal line showing a suggested cut threshold
fig_dendro_analysis.add_hline(
    y=color_threshold,
    line_dash="dash",
    line_color="red",
    annotation_text=f"Cut threshold ({color_threshold:.1f})",
    annotation_position="top right"
)

fig_dendro_analysis.update_layout(
    title='Hierarchical Clustering Dendrogram of 233 Category Centroids<br>(Ward Linkage, 50D PCA Features)',
    xaxis_title='Category',
    yaxis_title='Ward Distance',
    height=800,
    width=1400,
    xaxis={'tickangle': 90, 'tickfont': {'size': 6}}
)

fig_dendro_analysis.show()

# Analyze the structure at different cut heights
print("Macro-category counts at different cut heights:")
cut_heights = [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
max_height = max(linkage_matrix[:, 2])

for ratio in cut_heights:
    cut_height = ratio * max_height
    n_clusters = len(
        set(fcluster(linkage_matrix, cut_height, criterion='distance')))
    print(
        f"Cut at {ratio*100:.0f}% max height ({cut_height:.1f}): {n_clusters} clusters")

### 4.4 Category Centroid Distance Heatmap

To visualize how categories relate to each other in the 50D feature space we compute pairwise Euclidean distances between all 233 category centroids. The distance matrix is then reordered according to the hierarchical clustering leaf order, so that visually similar categories appear adjacent to each other. We display the first 100 categories to keep the heatmap readable.

In [ ]:
# Compute pairwise Euclidean distances between all 233 category centroids
centroid_distances = pdist(category_centroids, metric='euclidean')
distance_matrix = squareform(centroid_distances)

# Get the optimal leaf ordering from hierarchical clustering
# This reorders categories so that similar ones are adjacent
leaf_order = leaves_list(linkage_matrix)
ordered_categories = [categories[i] for i in leaf_order]

# Reorder the distance matrix according to clustering
ordered_distance_matrix = distance_matrix[np.ix_(leaf_order, leaf_order)]

# Show a zoomed version
n_zoom = 100
zoom_categories = ordered_categories[:n_zoom]

fig_heatmap_zoom = go.Figure(data=go.Heatmap(
    z=ordered_distance_matrix[:n_zoom, :n_zoom],
    x=zoom_categories,
    y=zoom_categories,
    colorscale='Viridis',
    reversescale=True,
    colorbar=dict(title='Euclidean Distance'),
    hovertemplate='%{x} ↔ %{y}<br>Distance: %{z:.3f}<extra></extra>'
))

fig_heatmap_zoom.update_layout(
    title=f'Zoomed Heatmap: First {n_zoom} Categories (Clustered Order)',
    xaxis=dict(tickangle=45, tickfont=dict(size=9)),
    yaxis=dict(tickfont=dict(size=9), autorange='reversed'),
    width=900,
    height=800
)

fig_heatmap_zoom.show()

The heatmap reveals the distance structure among the first 100 categories ordered by hierarchical clustering. The diagonal shows zero distance (yellow) as expected for self-comparisons. Several block structures are visible along the diagonal, indicating groups of categories with low inter-category distances (teal/green regions).

Notable patterns include a cluster of electronic/household objects in the upper-left (ipod, breadmaker, photocopier, microwave) and another group of elongated objects mid-section (chopsticks, tuning-fork, eyeglasses). The bottom-right corner shows a distinct block containing architectural structures (skyscraper, minaret, pyramid) with relatively low distances to each other but high distances (purple) to most other categories.

The predominant teal coloring across most of the matrix indicates moderate distances between categories, suggesting that while the CNN learned to distinguish broad visual concepts, many categories share enough visual features (shapes, textures, colors) to remain relatively close in feature space.

## 5. Supervised Baseline Classifier

Before evaluating unsupervised clustering, we establish a supervised baseline to measure how well the CNN features support classification when ground-truth labels are available. This upper bound helps us contextualize the clustering results: the gap between supervised accuracy and unsupervised purity reveals how much information is lost when labels are unavailable.

We use Logistic Regression because it's fast, interpretable, and effective with high-dimensional features. Unlike the CNN (which is a complex neural network), Logistic Regression learns simple linear decision boundaries in the 256-dimensional feature space. Strong performance here would confirm that the CNN successfully encoded discriminative visual information suitable for both classification and clustering tasks.

### 5.1 Train-Test Split with Stratification

We split our data into 80% training and 20% test sets using stratified sampling. Stratification ensures that each category maintains the same proportion in both sets.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    features_scaled,
    labels_loaded,
    test_size=0.2,
    random_state=42,
    stratify=labels_loaded
)

print(f"Training set size: {X_train.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")
print(f"Feature dimensions: {X_train.shape[1]}")
print(f"Number of classes: {len(np.unique(y_train))}")

### 5.2 Train Logistic Regression Classifier

We train multiple Logistic Regression configurations on the 256-dimensional CNN features to find the best balance between fitting the training data and generalizing to unseen examples:

| Configuration    | Description                                                   |
| ---------------- | ------------------------------------------------------------- |
| Baseline (C=1.0) | Default regularization strength                               |
| C=0.1            | Stronger L2 regularization to reduce overfitting              |
| C=0.1 + balanced | Regularization with class weighting for imbalanced categories |
| PCA-50D + C=0.1  | Regularization on dimensionality-reduced features             |

Key settings:

- max_iter=1000: Sufficient iterations for convergence with 233 classes
- solver='lbfgs': Efficient optimization for multinomial classification
- C parameter: Controls regularization strength (lower = stronger regularization)
- n_jobs=-1: Parallel processing across all CPU cores

In [ ]:
# type: ignore

# Baseline: default Logistic Regression
lr_baseline = LogisticRegression(
    max_iter=1000,
    random_state=42,
    solver='lbfgs',
    n_jobs=-1
)
lr_baseline.fit(X_train, y_train)

# Approach 1: Stronger regularization (C=0.1)
lr_regularized = LogisticRegression(
    max_iter=1000,
    random_state=42,
    solver='lbfgs',
    C=0.1,
    n_jobs=-1
)
lr_regularized.fit(X_train, y_train)

# Approach 2: Regularization + class balancing
lr_balanced = LogisticRegression(
    max_iter=1000,
    random_state=42,
    solver='lbfgs',
    C=0.1,
    class_weight='balanced',
    n_jobs=-1
)
lr_balanced.fit(X_train, y_train)

# Approach 3: PCA-reduced features (50D)
X_train_pca, X_test_pca, y_train_pca, y_test_pca = train_test_split(
    features_50d, labels_loaded, test_size=0.2, random_state=42, stratify=labels_loaded
)
lr_pca = LogisticRegression(
    max_iter=1000,
    random_state=42,
    solver='lbfgs',
    C=0.1,
    n_jobs=-1
)
lr_pca.fit(X_train_pca, y_train_pca)

# Collect results
results = []
configs = [
    ("Baseline (C=1.0)", lr_baseline, X_train, X_test, y_train, y_test),
    ("C=0.1", lr_regularized, X_train, X_test, y_train, y_test),
    ("C=0.1 + balanced", lr_balanced, X_train, X_test, y_train, y_test),
    ("PCA-50D + C=0.1", lr_pca, X_train_pca, X_test_pca, y_train_pca, y_test_pca),
]

for name, model, X_tr, X_te, y_tr, y_te in configs:
    train_acc = accuracy_score(y_tr, model.predict(X_tr))
    test_acc = accuracy_score(y_te, model.predict(X_te))
    y_pred = model.predict(X_te)
    p_macro, r_macro, f1_macro, _ = precision_recall_fscore_support(
        y_te, y_pred, average='macro', zero_division=0
    )
    p_weighted, r_weighted, f1_weighted, _ = precision_recall_fscore_support(
        y_te, y_pred, average='weighted', zero_division=0
    )
    results.append({
        'Configuration': name,
        'Train Acc': train_acc,
        'Test Acc': test_acc,
        'Macro F1': f1_macro,
        'Weighted F1': f1_weighted
    })

df_results = pd.DataFrame(results)
print(df_results.to_string(index=False))

# Select best model for downstream analysis
best_idx = df_results['Test Acc'].idxmax()
best_config = df_results.loc[best_idx, 'Configuration']

# Use best model for predictions
if 'PCA' in best_config:
    lr_classifier = lr_pca
    y_test_pred = lr_pca.predict(X_test_pca)
    y_train_pred = lr_pca.predict(X_train_pca)
    train_accuracy = accuracy_score(y_train_pca, y_train_pred)
    test_accuracy = accuracy_score(y_test_pca, y_test_pred)
elif 'balanced' in best_config:
    lr_classifier = lr_balanced
    y_test_pred = lr_balanced.predict(X_test)
    y_train_pred = lr_balanced.predict(X_train)
    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)
elif 'C=0.1' in best_config:
    lr_classifier = lr_regularized
    y_test_pred = lr_regularized.predict(X_test)
    y_train_pred = lr_regularized.predict(X_train)
    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)
else:
    lr_classifier = lr_baseline
    y_test_pred = lr_baseline.predict(X_test)
    y_train_pred = lr_baseline.predict(X_train)
    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)

# Final metrics for best model
precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(
    y_test if 'PCA' not in best_config else y_test_pca,
    y_test_pred, average='macro', zero_division=0
)
precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(
    y_test if 'PCA' not in best_config else y_test_pca,
    y_test_pred, average='weighted', zero_division=0
)

print(f"\nBest Model: {best_config}")
print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"\nMacro-averaged metrics:")
print(f"  Precision: {precision_macro:.4f}")
print(f"  Recall: {recall_macro:.4f}")
print(f"  F1-Score: {f1_macro:.4f}")
print(f"\nWeighted-averaged metrics:")
print(f"  Precision: {precision_weighted:.4f}")
print(f"  Recall: {recall_weighted:.4f}")
print(f"  F1-Score: {f1_weighted:.4f}")

| Configuration    | Train Acc | Test Acc  | Gap       | Macro F1  | Weighted F1 |
| ---------------- | --------- | --------- | --------- | --------- | ----------- |
| Baseline (C=1.0) | 80.7%     | 37.7%     | 43.0%     | 31.6%     | 37.3%       |
| **C=0.1**        | **63.8%** | **40.9%** | **22.9%** | **33.8%** | **39.4%**   |
| C=0.1 + balanced | 63.9%     | 39.2%     | 24.7%     | 33.3%     | 38.7%       |
| PCA-50D + C=0.1  | 46.7%     | 39.2%     | 7.5%      | 31.5%     | 37.1%       |

Best Model: C=0.1 (strongest regularization on full 256D features)

### 5.3 Per-Category Performance Analysis

We examine how classification accuracy varies across individual categories. This reveals which types of images are easiest or hardest to classify, and whether performance correlates with category size or visual distinctiveness.

In [ ]:
# Calculate per-category accuracy
per_class_accuracy = []
for i, category in enumerate(categories):
    mask = y_test == i
    if mask.sum() > 0:
        class_acc = (y_test_pred[mask] == i).sum() / mask.sum()
        per_class_accuracy.append({
            'category': category,
            'label_id': i,
            'accuracy': class_acc,
            'test_samples': mask.sum()
        })

df_class_acc = pd.DataFrame(per_class_accuracy)
df_class_acc_sorted = df_class_acc.sort_values('accuracy', ascending=False)

print("\nTop 10 Best Performing Categories:")
print(df_class_acc_sorted.head(10)[
      ['category', 'accuracy', 'test_samples']].to_string(index=False))

print("\nTop 10 Worst Performing Categories:")
print(df_class_acc_sorted.tail(10)[
      ['category', 'accuracy', 'test_samples']].to_string(index=False))

print(f"\nMean per-class accuracy: {df_class_acc['accuracy'].mean():.4f}")
print(f"Median per-class accuracy: {df_class_acc['accuracy'].median():.4f}")
print(f"Std per-class accuracy: {df_class_acc['accuracy'].std():.4f}")

fig_acc_dist = go.Figure()
fig_acc_dist.add_trace(go.Histogram(
    x=df_class_acc['accuracy'],
    nbinsx=30,
    name='Per-Class Accuracy'
))
fig_acc_dist.update_layout(
    title='Distribution of Per-Class Accuracy',
    xaxis_title='Accuracy',
    yaxis_title='Number of Classes',
    showlegend=False,
    height=500
)
fig_acc_dist.show()

| Best Categories | Accuracy | Test Samples | Worst Categories | Accuracy | Test Samples |
| --------------- | -------- | ------------ | ---------------- | -------- | ------------ |
| car-side        | 100.0%   | 21           | windmill         | 0%       | 17           |
| leopards        | 100.0%   | 35           | tuning-fork      | 0%       | 18           |
| faces-easy      | 98.7%    | 79           | rifle            | 0%       | 19           |
| motorbikes      | 98.6%    | 144          | dog              | 0%       | 19           |
| airplanes       | 97.9%    | 144          | mailbox          | 0%       | 17           |

Most categories cluster around 20-40% accuracy while a small subset achieves 80%+ accuracy. Two categories (car-side, leopards) achieve perfect 100% classification and several others exceed 90% (faces-easy, motorbikes, airplanes, sunflower, mars). These high-performing categories share distinctive visual signatures: consistent shapes, unique textures, or characteristic compositions that make them easily separable in feature space.

Categories that fail completely (0% accuracy) include everyday objects that appear in varied contexts (dog, goat, sneaker) or items with ambiguous visual features (tuning-fork, stirrups, spoon). These objects lack the consistent visual patterns needed for reliable classification.

| Statistic                 | Value  |
| ------------------------- | ------ |
| Mean per-class accuracy   | 34.52% |
| Median per-class accuracy | 30.43% |
| Standard deviation        | 23.90% |

The high standard deviation (23.9%) confirms dramatic performance variation across categories.

### 5.4 Supervised Classification on Macro-Categories

To further validate that the data-driven macro-categories represent meaningful visual groupings, we train a Logistic Regression classifier on macro-category labels instead of the original 233 fine-grained categories. If macro-categories capture coherent visual concepts, classification accuracy should improve significantly compared to the fine-grained baseline.

In [ ]:
# Train Logistic Regression on macro-categories at different granularities
macro_classification_results = []
macro_granularities = [5, 10, 15, 20, 30]


def get_macro_labels(linkage_matrix, n_macro, category_labels):
    """Map each image's category to its macro-category from dendrogram cut."""
    cat_to_macro = fcluster(linkage_matrix, n_macro, criterion='maxclust')
    return np.array([cat_to_macro[cat] - 1 for cat in category_labels])


for n_macro in macro_granularities:
    # Get macro-category labels for all images
    macro_labels_all = get_macro_labels(linkage_matrix, n_macro, labels_loaded)

    # Split data with stratification on macro-labels
    X_train_macro, X_test_macro, y_train_macro, y_test_macro = train_test_split(
        features_scaled,
        macro_labels_all,
        test_size=0.2,
        random_state=42,
        stratify=macro_labels_all
    )

    # Train Logistic Regression with best config (C=0.1)
    lr_macro = LogisticRegression(
        max_iter=1000,
        random_state=42,
        solver='lbfgs',
        C=0.1,  # Best configuration from Section 5.2
        n_jobs=-1
    )
    lr_macro.fit(X_train_macro, y_train_macro)

    # Evaluate
    train_acc = accuracy_score(y_train_macro, lr_macro.predict(X_train_macro))
    test_acc = accuracy_score(y_test_macro, lr_macro.predict(X_test_macro))

    macro_classification_results.append({
        'n_macro_categories': n_macro,
        'train_accuracy': train_acc,
        'test_accuracy': test_acc,
        'random_baseline': 1 / n_macro
    })

    print(
        f"Macro-categories: {n_macro:2d} | Train Acc: {train_acc:.4f} | Test Acc: {test_acc:.4f} | Random: {1/n_macro:.4f}")

df_macro_class = pd.DataFrame(macro_classification_results)

# Visualize results
fig_macro_class = go.Figure()
fig_macro_class.add_trace(go.Scatter(
    x=df_macro_class['n_macro_categories'],
    y=df_macro_class['test_accuracy'],
    mode='lines+markers',
    name='Test Accuracy',
    marker=dict(size=10)
))
fig_macro_class.add_trace(go.Scatter(
    x=df_macro_class['n_macro_categories'],
    y=df_macro_class['random_baseline'],
    mode='lines+markers',
    name='Random Baseline',
    line=dict(dash='dash')
))
fig_macro_class.add_hline(
    y=0.409,
    line_dash="dot",
    line_color="red",
    annotation_text="Fine-grained (233 classes): 40.9%",
    annotation_position="top right"
)
fig_macro_class.update_layout(
    title='Logistic Regression Accuracy on Data-Driven Macro-Categories (C=0.1)',
    xaxis_title='Number of Macro-Categories',
    yaxis_title='Test Accuracy',
    height=500,
    width=800
)
fig_macro_class.show()

As we reduce the number of target macro-categories from 30 to 5 test accuracy increases from 54% to 72%, showing that the CNN features support progressively better classification as the task becomes less granular. Even with just 5 broad visual groups, the classifier achieves 72% accuracy demonstrating that hierarchical clustering successfully identified meaningful visual structure. At all granularities, test accuracy remains far above the random baseline confirming that the learned features encode genuine visual patterns rather than noise. Reducing to 5-10 macro-categories achieves higher accuracy, validating our hierarchical clustering approach and suggesting that for practical applications with limited labeled data, training classifiers on 5-15 macro-categories is more effective than attempting fine-grained 233-way classification.

## 6. K-Means Clustering Analysis

Having established data-driven macro-categories through hierarchical clustering (Section 4.3), we now evaluate how well K-Means clustering aligns with these visual groupings. K-Means partitions images into k clusters by minimizing within-cluster variance, assigning each image to the nearest centroid. We measure cluster quality using purity: the fraction of images in each cluster that belong to the majority class. By evaluating purity against both the original 233 fine-grained categories and our hierarchically-derived macro-categories (5-30 groups), we can assess whether K-Means captures meaningful visual structure at different levels of granularity.

In [ ]:
# Run K-Means with different cluster counts and compute purity against macro-categories

cluster_counts = [10, 20, 50, 100, 233]


def calculate_purity(cluster_labels, true_labels):
    correct = sum(
        Counter(true_labels[cluster_labels == c]).most_common(1)[0][1]
        for c in np.unique(cluster_labels)
    )
    return correct / len(cluster_labels)


# Run K-Means and calculate purity for all configurations
all_purity_results = []
for k in cluster_counts:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    cluster_labels = kmeans.fit_predict(features_50d)
    fine_purity = calculate_purity(cluster_labels, labels_loaded)

    for n_macro in macro_granularities:
        macro_labels = get_macro_labels(linkage_matrix, n_macro, labels_loaded)
        macro_purity = calculate_purity(cluster_labels, macro_labels)
        all_purity_results.append({
            'n_kmeans_clusters': k,
            'n_macro_categories': n_macro,
            'fine_grained_purity': fine_purity,
            'macro_purity': macro_purity
        })

df_purity = pd.DataFrame(all_purity_results)

### 6.1 Purity Visualization

We visualize K-Means purity across two dimensions: the number of clusters (K) and the granularity of target categories. The line chart shows how purity changes as we move from fine-grained (233 categories) to coarser macro-categories (5-30 groups), while the heatmap provides a comprehensive view of all K × granularity combinations. Higher purity values (green) indicate that clusters are more homogeneous with respect to the target labels.

In [ ]:
# Purity line chart: fine-grained vs macro-categories
fig_purity = go.Figure()
for k in cluster_counts:
    subset = df_purity[df_purity['n_kmeans_clusters'] == k]
    x_vals = [233] + list(subset['n_macro_categories'])
    y_vals = [subset['fine_grained_purity'].iloc[0]] + \
        list(subset['macro_purity'])
    fig_purity.add_trace(go.Scatter(
        x=x_vals, y=y_vals, mode='lines+markers', name=f'K={k}',
        hovertemplate='%{x} categories<br>Purity: %{y:.4f}<extra></extra>'
    ))
fig_purity.update_layout(
    title='K-Means Purity: Fine-Grained (233) vs Data-Driven Macro-Categories',
    xaxis_title='Number of Target Categories (log scale)', yaxis_title='Cluster Purity',
    xaxis_type='log', xaxis=dict(tickvals=[5, 10, 15, 20, 30, 50, 100, 233]),
    height=500, width=900, legend_title='K-Means Clusters'
)
fig_purity.show()

# Purity heatmap
pivot = df_purity.pivot(index='n_kmeans_clusters',
                        columns='n_macro_categories', values='macro_purity')
pivot[233] = df_purity.groupby('n_kmeans_clusters')[
    'fine_grained_purity'].first()
pivot = pivot.sort_index(axis=1, ascending=False)

fig_heatmap = go.Figure(data=go.Heatmap(
    z=pivot.values, x=[str(c) for c in pivot.columns], y=[f'K={k}' for k in pivot.index],
    colorscale='RdYlGn', text=np.round(pivot.values, 3), texttemplate='%{text}',
    textfont={'size': 11}, colorbar=dict(title='Purity')
))
fig_heatmap.update_layout(
    title='Purity Heatmap: K-Means Clusters vs Number of Target Categories',
    xaxis_title='Number of Target Categories (Macro → Fine-grained)',
    yaxis_title='K-Means Clusters', height=400, width=800
)
fig_heatmap.show()

The visualizations reveal that K-Means clustering aligns well with broad visual groupings but struggles with fine-grained distinctions. At the fine-grained level (233 categories), purity ranges from just 10.6% (K=10) to 24.2% (K=233), confirming that unsupervised clustering cannot recover the original category labels. However, when evaluated against 5 macro-categories, all K values converge to 57-67% purity: a 3-6× improvement demonstrating that K-Means successfully captures the coarse visual structure discovered by hierarchical clustering.

The heatmap reinforces this pattern: the diagonal gradient from red (bottom-left, fine-grained with few clusters) to green (top-right, coarse macro-categories with many clusters) shows that purity increases as we either add more clusters or reduce the number of target categories. Notably, K=233 achieves only 24.2% purity on fine-grained labels compared to the supervised Logistic Regression's 40.9% accuracy: a gap that highlights the difficulty of unsupervised category recovery.

For practical applications, these results suggest that K-Means on CNN features provides a viable solution for broad categorization tasks (e.g., distinguishing animals from vehicles with ~65% purity at 5 macro-categories), while fine-grained classification (e.g., leopard vs. cheetah) requires supervised learning.